# GET your API from Here 
- https://aistudio.google.com/app/apikey

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(r"Resume.pdf")
data = loader.load_and_split()
data

[Document(page_content='S. DINESH KUMAR || 2021PECAI250 \nsvani4830@gmail.com ||+91 9342372124 ||https://github.com/SDineshKumar1304 || www.linkedin.com/in/s-dineshkumar2004  \nPROFESSIONAL EXPERIENCE \nINNOMATICS RESEARCH LABS                                                                                                India, Hyderabad \nJunior Data Scientist Intern                                                                                             Jan 2024-Present (Remote) \nML Flow Model Experimentation: \nApplication Development: \nExploratory Data Analysis: \n● Developed ” Gen-AI A Multi-faceted AI Endeavor ”.Working on Gemini-1.5 pro  LLM  \n● Utilized Chroma DB  to develop a Search Engine  using tfidf-vectorizer and Cosine similarity  model for \nMovie Subtitles information retrieval system. \n● Developed Real Time Sentiment Analysis on Flipkart Products reviews web application with Flask \nDeployed the Web application on AWS \n● Utilized ML Flow and Prefect Orchestrati

In [2]:
from langchain_text_splitters import NLTKTextSplitter

text_splitter = NLTKTextSplitter(chunk_size=200, chunk_overlap=200)

chunks = text_splitter.split_documents(data)

print(len(chunks))

print(type(chunks[0]))

Created a chunk of size 784, which is longer than the specified 200
Created a chunk of size 227, which is longer than the specified 200
Created a chunk of size 280, which is longer than the specified 200
Created a chunk of size 370, which is longer than the specified 200
Created a chunk of size 221, which is longer than the specified 200


39
<class 'langchain_core.documents.base.Document'>


In [3]:

from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_model = GoogleGenerativeAIEmbeddings(google_api_key="YOUR_API_KEY", 
                                               model="models/embedding-001")


In [8]:
pip install chromadb

     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ------------------ --------------------- 30.7/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/526.8 kB ? eta -:--:--
   ----- ---------------------------------- 71.7/526.8 kB 2.0 MB/s eta 0:00:01
   ----------- ---------------------------- 153.6/526.8 kB 1.8 MB/s eta 0:00:01
   ---------------- --------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.27.0 which is incompatible.
langchain 0.0.340 requires langsmith<0.1.0,>=0.0.63, but you have langsmith 0.1.53 which is incompatible.
spacy 3.6.1 requires typer<0.10.0,>=0.3.0, but you have typer 0.12.3 which is incompatible.


In [4]:
# Embed each chunk and load it into the vector store
db = Chroma.from_documents(chunks, embedding_model, persist_directory="./chroma_db_")

# Persist the database on drive
db.persist()

C:\Users\svani\anaconda3\envs\tensorflow\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [5]:
db_connection = Chroma(persist_directory="./chroma_db_", embedding_function=embedding_model)

In [6]:
retriever = db_connection.as_retriever(search_kwargs={"k": 1})

print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [7]:
user_input = 'Explain this Resume'
retrieved_docs = retriever.invoke(user_input)

In [8]:
print(retrieved_docs[0].page_content)

ADDITIONAL INFORMATION 
● Technical Skills: Python,Flask,Streamlit,SQL&DBMS,EDA,MLOPS,Machine Learning,Data 
Analytics,PowerBI, NLP, AWS Deployment.


In [9]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are a Helpful AI Bot. 
    You take the context and question from user. Your answer should be based on the specific context."""),
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Aswer the question based on the given context.
    Context:
    {context}
    
    Question: 
    {question}
    
    Answer: """)
])


In [10]:
from langchain_core.runnables import RunnablePassthrough

In [11]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

chat_model = ChatGoogleGenerativeAI(google_api_key="YOUR_API_KEY", 
                                   model="gemini-1.5-pro-latest")

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_template
    | chat_model
    | output_parser
)

In [16]:
response = rag_chain.invoke("Can you tell me about job?")

response

'## Potential Job Roles Based on Your Skills:\n\nGiven your technical skills in Python, Flask, Streamlit, SQL & DBMS, EDA, MLOPS, Machine Learning, Data Analytics, PowerBI, NLP, and AWS Deployment, you have a strong foundation for several exciting job roles:\n\n**Data Science & Machine Learning:**\n\n* **Data Scientist:** Analyze large datasets, build and deploy machine learning models, and derive insights to solve business problems.\n* **Machine Learning Engineer:** Focus on the development and deployment of machine learning models, including model optimization and scaling.\n* **NLP Engineer:** Develop natural language processing applications, such as chatbots, text summarization tools, and sentiment analysis systems.\n\n**Data Analytics & Business Intelligence:**\n\n* **Data Analyst:** Collect, clean, and analyze data to identify trends, patterns, and insights, and create reports and visualizations to communicate findings.\n* **Business Intelligence Analyst:** Develop and maintain BI

In [17]:
from IPython.display import Markdown as md

md(response)

## Potential Job Roles Based on Your Skills:

Given your technical skills in Python, Flask, Streamlit, SQL & DBMS, EDA, MLOPS, Machine Learning, Data Analytics, PowerBI, NLP, and AWS Deployment, you have a strong foundation for several exciting job roles:

**Data Science & Machine Learning:**

* **Data Scientist:** Analyze large datasets, build and deploy machine learning models, and derive insights to solve business problems.
* **Machine Learning Engineer:** Focus on the development and deployment of machine learning models, including model optimization and scaling.
* **NLP Engineer:** Develop natural language processing applications, such as chatbots, text summarization tools, and sentiment analysis systems.

**Data Analytics & Business Intelligence:**

* **Data Analyst:** Collect, clean, and analyze data to identify trends, patterns, and insights, and create reports and visualizations to communicate findings.
* **Business Intelligence Analyst:** Develop and maintain BI dashboards and reports, using tools like PowerBI, to track key performance indicators and support data-driven decision making.

**Software Development & MLOps:**

* **Backend Developer:** Build and maintain the server-side logic of web applications using Python and Flask.
* **MLOps Engineer:** Design and implement MLOps pipelines to automate the machine learning model lifecycle, ensuring efficient and reliable model deployment.

**Additional Potential Roles:**

* **Research Scientist:** Conduct research in areas like machine learning, NLP, or artificial intelligence, and contribute to the advancement of the field.
* **Data Engineer:** Design, build, and maintain data infrastructure, including data pipelines and databases, to support data-driven applications.

**Choosing the Right Path:**

* **Interests:** Consider which areas within data science, machine learning, and analytics you find most engaging.
* **Experience:** Leverage your existing experience and skills to identify roles that best align with your background.
* **Industry:** Explore different industries, such as technology, finance, healthcare, or e-commerce, to find one that interests you.

**Further Development:**

* **Stay updated:** Continuously learn and adapt to new technologies and trends in the field.
* **Build a portfolio:** Showcase your skills and projects through a portfolio website or GitHub repository.
* **Network:** Connect with professionals in your desired field to learn about job opportunities and industry insights.

I hope this information helps! Let me know if you have any further questions. 


In [ ]:
# END